# Vizzuality Challenge, ETL pipeline. 
 Summarized total ecosystem carbon of the northern lakes region in the USA using data from the National Forest Carbon Monitoring System.

This Notebook is set to process de downloaded files from the National Forest Carbon Monitoring System (Check Readme.txt for specifications on download.py)

This ETL pipeline will: 

- 1 - Automatically download the necesary data (download.py)

- 2 - Select the ROI (Every county of the states of Michigan, Wisconsin and Minnesota) within the USA administrative boundaries Shapefile source

- 3 - Summarize Total Ecosystem Carbon for the set ROI. TotalExosystemCarbon_2020 is the choosed raster due that is the "current" state of carbon ecosystem data in the region. 

- 4 - Convert according to necesity CRSs, units, etc. 

- 5 - Create a .gpkg file with the Total Ecosystem Carbon values

- 6 - Upload it to a relational database for inquires 

- 7 - Simple vizzualization of the output data. 

In [ ]:
import sys
print(sys.executable)

d:\conda_envs\vizz_challenge\python.exe


In [2]:
from pathlib import Path
import requests, zipfile
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterstats import zonal_stats
import pandas as pd
import numpy as np
import warnings

# Base directories. Creates the necessary folder structure for simplicity
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data"
RAW = DATA_DIR / "raw"
PROCESSED = DATA_DIR / "processed"
OUTPUT = DATA_DIR / "output"

for folder in [RAW, PROCESSED, OUTPUT]:
    folder.mkdir(parents=True, exist_ok=True)

print("Folders are ready:", list(DATA_DIR.glob('*')))

Folders are ready: [WindowsPath('d:/Vizzuality_challenge/data/.gitkeep'), WindowsPath('d:/Vizzuality_challenge/data/output'), WindowsPath('d:/Vizzuality_challenge/data/processed'), WindowsPath('d:/Vizzuality_challenge/data/raw')]


In [4]:
# Data download, quick import from download.py file. You can also execute .py file directly if wanted.
# This module is added to the notebook for the intent of traceability. 

from download import download_and_extract  # 👈 importa la función desde tu script

# URLs
COUNTIES_URL = "https://www2.census.gov/geo/tiger/TIGER2025/COUNTY/tl_2025_us_county.zip"
CARBON_URL = "https://usfs-public.box.com/shared/static/v861gwms9fq68sitl0r3vs2v3moxeu9x.zip"

# Descarga
counties_dir = download_and_extract(COUNTIES_URL, RAW)
carbon_dir = download_and_extract(CARBON_URL, RAW)

# Encontrar paths después de la descarga
shp_path = list(counties_dir.rglob("*.shp"))[0]
raster_path = next(carbon_dir.rglob("*.tif"))
print("✅ Descargas completas.")


Skipping: tl_2025_us_county already exists with content.
Skipping: v861gwms9fq68sitl0r3vs2v3moxeu9x already exists with content.
✅ Descargas completas.


In [5]:
# County filtering 

gdf = gpd.read_file(shp_path)

# FIPS de Michigan (26), Minnesota (27), Wisconsin (55) (.shp column)
filtered = gdf[gdf["STATEFP"].isin(["26", "27", "55"])].copy()
filtered_path = PROCESSED / "counties_MI_WI_MN.gpkg" # Change to gpkg for file consistency
filtered.to_file(filtered_path, driver="GPKG")

print(f"Shapefile saved in: {filtered_path}")
print("County count", len(filtered))

Shapefile saved in: d:\Vizzuality_challenge\data\processed\counties_MI_WI_MN.gpkg
County count 242


In [9]:
# Raster and poligons CRS corrections. Normalization of objects. 
# Module obtains CRS from raster, equals it to .gpkg file and defines route to be used by Zonal Stats. 

print("\nRaster & counties correction")
print("Original raster path:", raster_path)

with rasterio.open(raster_path) as src:
    raster_crs = src.crs
    
    # 1. Reproyects de .gpkg file to equal rasters CRS. 
    gdf_reproj = gpd.read_file(filtered_path).to_crs(raster_crs)
    
    # 2. Saves the reproyected .pgpk for future use
    filtered_path_reproj = PROCESSED / "counties_reproj.gpkg"
    gdf_reproj.to_file(filtered_path_reproj, driver="GPKG")
    
    clipped_raster = raster_path # Due to RAM limitations...  

print(f"New Geopackage saved in: {filtered_path_reproj}")
print(f"⚠️ The variable 'clipped_raster' points to the original raster ({clipped_raster.name}) to avoid memory errors.")



Raster & counties correction
Original raster path: d:\Vizzuality_challenge\data\raw\v861gwms9fq68sitl0r3vs2v3moxeu9x\Data\NLS\NLS_AbovegroundBiomass2020.tif
New Geopackage saved in: d:\Vizzuality_challenge\data\processed\counties_reproj.gpkg
⚠️ The variable 'clipped_raster' points to the original raster (NLS_AbovegroundBiomass2020.tif) to avoid memory errors.


In [ ]:
# Raster units conversion, Zonal stats, and output export

print("\nCalculating total carbon per county...")

# Converts Units in original raster Mg CO₂e/acre to Mg C/pixel. Needed to Sum Total Carbon per county. 

# FACTOR = (Mg CO₂e/acre to Mg C/pixel)
FACTOR = (900 / 4046.86) * (12 / 44) 
NODATA_ORIGINAL = 65535 # Raster's Nodata value
MAX_VALUE = 173 # Maximum clipping value used in the original analysis. Just a error check point.

def sum_cleaned_raw(a):
    """Sum raw pixels (Mg CO₂e/acre) after NoData cleaning and clipping."""
    
    # FIX: Convert to float32 to allow np.nan and perform float calculations
    a = a.astype(np.float32) 
    
    # 1. Cleaning
    a[a == NODATA_ORIGINAL] = np.nan # Apply NoData
    a[a > MAX_VALUE] = np.nan        # Removes inflated values if they exist. 
    
    # 2. Total sum of clean raw values
    return np.nansum(a)

def count_valid_pixels(a):
    """Count the number of valid pixels after cleaning."""
    # Just another checkpoint, had some trouble "Relying" on output values. 
    a = a.astype(np.float32) 
    a[a == NODATA_ORIGINAL] = np.nan
    a[a > MAX_VALUE] = np.nan
    return np.nansum(np.isfinite(a))


# Reads reprojected polygons
gdf = gpd.read_file(filtered_path_reproj) 

# ---- ZONAL STATISTICS -----
print(" Converting units and running zonal stats...")

# Temporarily suppress rasterstats warnings due to nodata in custom stats. had some
# issues with noData values, found this way to prevent it from processing data.
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    stats = zonal_stats(
        vectors=gdf,
        raster=str(clipped_raster), # Use path to ORIGINAL raster
        stats=[], 
        add_stats={'raw_sum_CO2e_acre': sum_cleaned_raw, 'valid_pixels': count_valid_pixels},
        geojson_out=False,
        nodata=NODATA_ORIGINAL, 
        all_touched=False
    )

# Add columns
gdf["raw_sum_CO2e_acre"] = [s["raw_sum_CO2e_acre"] for s in stats]
gdf["valid_pixels"] = [s["valid_pixels"] for s in stats]

# Conversion of units to Mg C
print("Applying conversion factor (Mg CO₂e/acre to Mg C)...")
gdf["total_MgC"] = gdf["raw_sum_CO2e_acre"] * FACTOR

# Removes columns we don't use/want
columns_to_drop = [
    "COUNTYNS", "GEOIDFQ", "LSAD", "MTFCC", "CSAFP",
    "METFIVFP", "FUNCSTAT", "ALAND", "AWATER", "INTPTLAT", "INTPTLON"
]

# Only drop columns that actually exist (avoids KeyError)
gdf = gdf.drop(columns=[c for c in columns_to_drop if c in gdf.columns])

print(f"🧹 Dropped columns: {[c for c in columns_to_drop if c not in gdf.columns]}")

# --- Step 5: Export result ---
out_gpkg = OUTPUT / "carbon_total_by_county.gpkg"
gdf.to_file(out_gpkg, driver="GPKG")

print(f"\n💾 GeoPackage saved in: {out_gpkg}")
print("\n📊 Preview of results:")
print(gdf[["NAME", "raw_sum_CO2e_acre", "valid_pixels", "total_MgC"]].head())
print(f"\n✅ Process completed successfully. Total counties processed: {len(gdf)}")




Calculating total carbon per county...
⚙️ Running zonal_stats with cleaning functions...
⚙️ Applying conversion factor (Mg CO₂e/acre to Mg C)...
🧹 Dropped columns: ['COUNTYNS', 'GEOIDFQ', 'LSAD', 'MTFCC', 'CSAFP', 'METFIVFP', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON']

💾 GeoPackage saved in: d:\Vizzuality_challenge\data\output\carbon_total_by_county.gpkg

📊 Preview of results:
         NAME  raw_sum_CO2e_acre  valid_pixels     total_MgC
0    Richland         80253840.0        908127  4.867643e+06
1       Grant          4373964.0         67423  2.652944e+05
2  Green Lake         18854288.0        275296  1.143571e+06
3    Walworth         25412840.0        317963  1.541367e+06
4       Eaton         35590720.0        440851  2.158687e+06

✅ Process completed successfully. Total counties processed: 242
